# Collaboration and Competition

---

In [1]:
# Declare agent
import sys

# Add the subfolder to sys.path
sys.path.append('./tennis')

from unityagents import UnityEnvironment
import numpy as np
import importlib
import model
import agent
importlib.reload(agent)
importlib.reload(model)

# env = UnityEnvironment(file_name="Reacher_Windows_x86_64/Reacher.exe")
env = UnityEnvironment(file_name="tennis/Tennis.app")
# env = UnityEnvironment(file_name="reacher/Reacher_Multi.app")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))


# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)
# declare the agent
config = agent.MultiAgentPPOConfig(
    num_state = state_size,
    num_action = action_size,
    num_agent = num_agents,
    actor_learning_rate=1e-4,
    critic_learning_rate=1e-4,
    batch_size=64,
    clip_eps=0.2,
    entropy_coef=0.01,
    gamma = 0.99,
    lam = 0.95,
    replay_buffer_size=1000,
    weight_decay=0,
    ppo_epochs=2)

print("declaring agent")
agent = agent.MultiAgentPPO(config)
print("declaring agent done")

output_folder_path = "tennis/weights/"


Mono path[0] = '/Users/nuocheng/Desktop/Reinforcement_Learning/Reinforcement-Learning-Projects/tennis/Tennis.app/Contents/Resources/Data/Managed'
Mono config path = '/Users/nuocheng/Desktop/Reinforcement_Learning/Reinforcement-Learning-Projects/tennis/Tennis.app/Contents/MonoBleedingEdge/etc'


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Size of each action: 2
There are 2 agents. Each observes a state with length: 24
Number of agents: 2
declaring agent
declaring agent done


In [2]:
# Function to train the network
from collections import deque
import matplotlib.pyplot as plt

def train(agent, env, num_agents, folder_path, n_episodes=2000, max_t=1000):
    rewards_window = deque(maxlen=100)  # last 100 scores

    for episode in range(n_episodes):
        # Reset the environment in train mode.
        env_info = env.reset(train_mode=True)[brain_name]
        # Get initial observations for all agents.
        states = env_info.vector_observations[0:num_agents]
        episode_rewards = np.zeros(num_agents)
        done = False
        step = 0
        while not done and step < max_t:
            # Select actions and get log probabilities.
            actions, log_probs = agent.act(states)
            
            # Send the actions to the environment.
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations[0:num_agents]
            rewards = env_info.rewards[0:num_agents]
            dones = env_info.local_done[0:num_agents]
            
            # Store the transition in the buffer.
            agent.store(states, actions, rewards, next_states, dones, log_probs)
            
            # Update the current state.
            states = next_states
            episode_rewards += np.array(rewards)
            step += 1
            
            if any(dones):
                done = True
        
        # Update networks after the episode (if enough samples are collected).
        update_results = agent.learn()
        rewards_window.append(np.max(episode_rewards))
        if update_results is not None:
            critic_loss, actor_loss = update_results
            print("Episode {}: Curr Rewards: {} | Avg Rewards: {} | Critic Loss: {:.4f} | Actor Loss: {:.4f}".format(
                episode, np.max(episode_rewards), np.mean(rewards_window), critic_loss, actor_loss))
        # else:
        #     print("Episode {}: Rewards: {} (Not enough samples to update)".format(episode, episode_rewards))
    

# scores, avg_scores = 
train(agent, env, num_agents, output_folder_path, 50000, 1000)
env.close()

# # save the trained weight
# agent.save_model(output_folder_path)

# # plot the score distribution
# fig = plt.figure()
# ax = fig.add_subplot(111)
# plt.plot(np.arange(len(scores)), scores, color = 'blue', label = 'Score Each Episode')
# plt.plot(np.arange(len(avg_scores)), avg_scores, color = 'red', label = 'Average Score')
# # Optionally, set the y-axis limits to cover both data sets
# plt.ylabel('Rewards')
# plt.xlabel('Episode #')
# plt.show()

Episode 4: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0002 | Actor Loss: 0.0135
Episode 9: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0002 | Actor Loss: 0.0151
Episode 14: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0001 | Actor Loss: 0.0179
Episode 19: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0001 | Actor Loss: 0.0157
Episode 24: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0001 | Actor Loss: 0.0179
Episode 29: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0001 | Actor Loss: 0.0180
Episode 34: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0000 | Actor Loss: 0.0166
Episode 39: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0000 | Actor Loss: 0.0164
Episode 44: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0000 | Actor Loss: 0.0154
Episode 48: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0000 | Actor Loss: 0.0157
Episode 52: Curr Rewards: 0.0 | Avg Rewards: 0.0 | Critic Loss: 0.0000 | Actor Los

KeyboardInterrupt: 

In [ ]:
# Test the model
# Declare agent
import sys

# Add the subfolder to sys.path
sys.path.append('./reacher')

from unityagents import UnityEnvironment
import numpy as np
import importlib
import model
import agent
importlib.reload(agent)
importlib.reload(model)

# env = UnityEnvironment(file_name="Reacher_Windows_x86_64/Reacher.exe")
env = UnityEnvironment(file_name="reacher/Reacher_Multi.app")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))


# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)
# declare the agent
config = agent.DDPGConfig(
    num_state = state_size,
    num_action = action_size,
    num_agent = num_agents,
    actor_learning_rate=1e-4,
    critic_learning_rate=1e-4,
    batch_size=512,
    update_every_timestamp=20,
    update_time_each_stamp=20,
    discount_factor=0.99,
    replay_buffer_size=1e6,
    weight_decay=0)
output_folder_path = "tennis/weights/"
agent = agent.DDPGAgent(config)
agent.load_model(output_folder_path)

env_info = env.reset(train_mode=False)[brain_name] # reset the environment
states = env_info.vector_observations              # get the current state
cnt  = 1
for _ in range(3000):
    print(cnt)
    cnt+=1
    actions = agent.act(states)
    env_info = env.step(actions)[brain_name]
    states = env_info.vector_observations
    if np.any(env_info.local_done):
        break
env.close()

